In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import pickle

pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

In [3]:
disease_code = 309

lab_case = pd.read_csv('/home/data/sensitive_disease/{}_Lab.csv'.format(disease_code))
med_case = pd.read_csv('/home/data/sensitive_disease/{}_Med.csv'.format(disease_code))
procedure_case = pd.read_csv('/home/data/sensitive_disease/{}_Procedure.csv'.format(disease_code))
patient_case = pd.read_csv('/home/data/sensitive_disease/{}_Patient.csv'.format(disease_code))

print lab_case.shape
print med_case.shape
print procedure_case.shape
print patient_case.shape

lab_control = pd.read_csv('/home/data/sensitive_disease/nonSD5pct_Lab.csv')
med_control = pd.read_csv('/home/data/sensitive_disease/nonSD5pct_Med.csv')
procedure_control = pd.read_csv('/home/data/sensitive_disease/nonSD5pct_Procedure.csv')
patient_control = pd.read_csv('/home/data/sensitive_disease/nonSD5pct_Patient.csv')

print lab_control.shape
print med_control.shape
print procedure_control.shape
print patient_control.shape

/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (19,27,29,30,31,32,33,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(13429214, 29)
(1525585, 49)
(8586, 5)


/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6,19,26,27,29,30,31,32,33,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(7342024, 29)
(814389, 49)
(1346337, 25)
(32917, 2)


In [5]:
patient_case_person_id = set(med_case['person_id']) | set(lab_case['person_id']) | set(procedure_case['person_id'])

import random
random.seed(1)
patient_case_sample = random.sample(patient_case_person_id, 5000)

med_case_sample = med_case[med_case.person_id.isin(patient_case_sample)]
lab_case_sample = lab_case[lab_case.person_id.isin(patient_case_sample)]
procedure_case_sample = procedure_case[procedure_case.person_id.isin(patient_case_sample)]

## lab features

In [8]:
name_mapping = pd.read_csv('name_mapping/lab_name_mapping.csv')
name_mapping = dict(zip(name_mapping.irregular_name.str.lower(), 
                        name_mapping.regular_name.str.lower()))

def normalize_lab_nm(lab_name):
    norm_name = lab_name.strip().lower().replace(', ', ' ').replace(' - ', ' ')
    if norm_name in name_mapping:
        return name_mapping[norm_name]
    else:
        return norm_name

def select_rows(df):
    return (  (df.lab_val < 9999999.0).values
            & (df.lab_ref_high_val_numeric < 9999999.0).values
            & (df.lab_ref_low_val_numeric < 9999999.0).values
           )

lab_case_sample = lab_case_sample.assign(
    lab_nm_lower_case=lab_case_sample['lab_nm'].apply(normalize_lab_nm),
    lab_ref_high_val_numeric=lab_case_sample['lab_ref_high_val'].apply(pd.to_numeric, errors='ignore'),
    lab_ref_low_val_numeric=lab_case_sample['lab_ref_low_val'].apply(pd.to_numeric, errors='ignore')
)

lab_count_by_nm = pd.DataFrame(lab_309_sample.loc[select_rows].groupby('lab_nm_lower_case').size(), 
                               columns=['count'])
lab_case_frequent_nm = set(lab_count_by_nm[lab_count_by_nm['count'] > 50].index)

print 'lab {} features: '.format(disease_code)
print len(lab_309_frequent_nm)


lab_nonSD['lab_nm_lower_case'] = lab_nonSD['lab_nm'].apply(normalize_lab_nm)
lab_count_by_nm = pd.DataFrame(lab_nonSD.loc[select_rows].groupby('lab_nm_lower_case').size(), 
                               columns=['count'])
lab_nonSD_frequent_nm = set(lab_count_by_nm[lab_count_by_nm['count'] > 50].index)

print 'lab non SD features: '
print len(lab_nonSD_frequent_nm)

print 'lab features: '
lab_frequent_nm = lab_nonSD_frequent_nm | lab_309_frequent_nm
print len(lab_frequent_nm)

/usr/lib/python2.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


lab 309 features: 
345
lab non SD features: 
534
lab features: 
570


## medication features

In [13]:
from collections import OrderedDict

name_mapping = pd.read_csv('name_mapping/med_name_mapping.csv')
name_mapping = dict(zip(name_mapping.irregular_name.str.lower(), 
                        name_mapping.regular_name.str.lower()))

def normalize_med_nm(name):
    norm_name = name.strip().lower().replace(', ', ' ').replace(' - ', ' ').replace('/', '-') \
                    .replace(' topical', '').replace(' ophthalmic', '').replace(' (product)', '')\
                    .replace(' (substance)', '').replace(' product', '').replace(' [chemical-ingredient]', '')

    norm_name = '-'.join(OrderedDict.fromkeys(norm_name.split('-')))
    
    if norm_name in name_mapping:
        return name_mapping[norm_name]
    else:
        return norm_name

med_case_sample = med_case_sample.assign(
    generic_nm_lower_case=med_case_sample['generic_nm'].apply(normalize_med_nm)
)
med_count_by_nm = pd.DataFrame(med_case_sample.groupby('generic_nm_lower_case').size(), 
                               columns=['count'])
med_case_frequent_nm = set(med_count_by_nm[med_count_by_nm['count'] > 50].index)

print 'med {} features: '.format(disease_code)
print len(med_case_frequent_nm)

    
med_control = med_control.assign(
    generic_nm_lower_case=med_control['generic_nm'].apply(normalize_med_nm)
)
med_count_by_nm = pd.DataFrame(med_control.groupby('generic_nm_lower_case').size(), 
                               columns=['count'])
med_control_frequent_nm = set(med_count_by_nm[med_count_by_nm['count'] > 50].index)

print 'non SD features: '
print len(med_control_frequent_nm)

print 'total features:'
med_frequent_nm = med_case_frequent_nm | med_control_frequent_nm
print len(med_frequent_nm)


med 309 features: 
774
non SD features: 
775
total features:
885


## Procedure features

In [6]:
name_mapping = pd.read_csv('procedure_309_name_mapping.csv')
name_mapping = dict(zip(name_mapping.irregular_name.str.lower(), 
                        name_mapping.regular_name.str.lower()))

def normalize_order_nm(name):
    norm_name = name.strip().lower().replace(', ', ' ').replace(' - ', ' ')
    if norm_name.endswith(' nmh'):
        norm_name = norm_name[:-4]
    elif norm_name.endswith(' nmff'):
        norm_name = norm_name[:-5]
        
    if norm_name in name_mapping:
        return name_mapping[norm_name]
    else:
        return norm_name

def select_rows(df):
    return (  (df.order_type_desc != 'LAB').values
            & (df.order_type_desc != 'VASCULAR LAB').values
           )

procedure_309_sample['order_nm_lower_case'] = procedure_309_sample['order_nm'].apply(normalize_order_nm)
procedure_count_by_nm = pd.DataFrame(procedure_309_sample.loc[select_rows].groupby('order_nm_lower_case').size(), 
                                     columns=['count'])
procedure_309_frequent_nm = set(procedure_count_by_nm[procedure_count_by_nm['count'] > 50].index) 

print 'procedure 309 features: '
print len(procedure_309_frequent_nm)


procedure_nonSD['order_nm_lower_case'] = procedure_nonSD['order_nm'].apply(normalize_order_nm)
procedure_count_by_nm = pd.DataFrame(procedure_nonSD.loc[select_rows].groupby('order_nm_lower_case').size(), 
                                     columns=['count'])
procedure_nonSD_frequent_nm = set(procedure_count_by_nm[procedure_count_by_nm['count'] > 50].index) 

print 'procedure non SD features: '
print len(procedure_nonSD_frequent_nm)

print 'procedure features: '
procedure_frequent_nm = procedure_309_frequent_nm | procedure_nonSD_frequent_nm
print len(procedure_frequent_nm)

/usr/lib/python2.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


procedure 309 features: 
558
procedure non SD features: 
729
procedure features: 
782


### find common names in case and control

In [9]:
!ls *.txt -l

-rw-rw-r-- 1 yzx3653 yzx3653   708 Jul 23 22:35 309_labs_case.txt
-rw-rw-r-- 1 yzx3653 yzx3653  4690 Jul 23 22:36 309_labs_common.txt
-rw-rw-r-- 1 yzx3653 yzx3653  3898 Jul 23 22:34 309_labs_control.txt
-rw-rw-r-- 1 yzx3653 yzx3653  1766 Jul 23 22:35 309_meds_case.txt
-rw-rw-r-- 1 yzx3653 yzx3653 11006 Jul 23 22:36 309_meds_common.txt
-rw-rw-r-- 1 yzx3653 yzx3653  2830 Jul 23 22:34 309_meds_control.txt
-rw-rw-r-- 1 yzx3653 yzx3653  1286 Jul 23 22:35 309_procedures_case.txt
-rw-rw-r-- 1 yzx3653 yzx3653 13172 Jul 23 22:36 309_procedures_common.txt
-rw-rw-r-- 1 yzx3653 yzx3653  6033 Jul 23 22:34 309_procedures_control.txt


In [10]:
lab_common_nm = '\n'.join(sorted(lab_nonSD_frequent_nm & lab_309_frequent_nm))

with open('309_labs_common.txt', 'w') as fp:
    fp.write(lab_common_nm)

!cat 309_labs_common.txt

%cd3
abs cd3+
abs cd3+ cd4+
abs cd3+ cd8+
absolute bands
absolute basophils
absolute blast
absolute cd3
absolute cd3+
absolute cd3/cd4
absolute cd3/cd8
absolute eosinophils
absolute immature granulocytes
absolute lymphocytes
absolute metamyelocyte
absolute monocyte
absolute myelocyte
absolute neutrophil bands
absolute neutrophil metamyelocytes
absolute neutrophil myelocytes
absolute neutrophil promyelocytes
absolute neutrophils
absolute plasma cell
absolute promyelocyte
absolute segs
adrenocorticotropic hormone
agap
albumin
albumin (spep)
albumin glob
alkaline phosphatase
alpha fetoprotein tumor
alt (sgpt)
ammonia
angiotensin i converting enzyme
angle native blood
anti-thyroglobulin antibody
anticardiolipin igg
anticardiolipin igm
antithrombin iii
antithyroglobulin
arterial base ex/def
arterial bicarbonate
arterial pco2
arterial ph
arterial po2
arterial total hemoglobin
ast (sgot)
b-natriuretic peptide/bnp
band
baso#
baso%
basophil
bedside glucose
beta 2 microglobulin
bicarbonate
bilir

In [11]:
med_common_nm = '\n'.join(sorted(med_nonSD_frequent_nm & med_309_frequent_nm))

with open('309_meds_common.txt', 'w') as fp:
    fp.write(med_common_nm)

!cat 309_meds_common.txt

-furo(3',4':6,7)naphtho(2,3-d)-glucopyranosyl)oxy)-1,3-5,8,8a,9-tetrahydro-dioxol-3,5-ethylidene-one 9-5-6(5ah)-dimethoxyphenyl)-(5alpha,5abeta,8aalpha,9beta(r*)))-(4-beta-d-o- (5r-((4,6-hydroxy
-methyl-nitro-(2-1,1-2-furanyl)methyl)thio)ethyl)-ethenediamine n-((dimethylamino)methyl)-(((5-n'
3 acid ethyl esters (usp)-omega
3 fatty acids-omega
abatacept
acetaminophen
acetazolamide
acetylcysteine
activated dimethicone
activated methylpolysiloxane
acyclovir
adalimumab
adapalene
adenosine
adol-tetanus-pertuss (tdap) adult-diphth
aflibercept
albumin human usp
albuterol
albuterol-ipratropium
alclometasone
alendronate
alendronic acid
alfentanil
alfuzosin
allopurinol
alprazolam
alteplase
amantadine
amikacin
aminocaproic acid-6
aminophylline
amiodarone
amitriptyline
amlodipine
amlodipine-atorvastatin
amlodipine-benazepril
amoxicillin
amoxicillin-clavulanate
amphetamine-dextroamphetamine
amphotericin b liposome
ampicillin
amylases-lipase-endopeptidases
anastrozole
apixaban
aprepitant
aripiprazol

In [12]:
procedure_common_nm = '\n'.join(sorted(procedure_nonSD_frequent_nm & procedure_309_frequent_nm))

with open('309_procedures_common.txt', 'w') as fp:
    fp.write(procedure_common_nm)

!cat 309_procedures_common.txt

acoustic reflex testing
adenosine (vasodilator) perfusion study
allergy patch tests
allergy prick tests for aeros
allergy(intradermal)skin tests
ambulatory bp monitoring (24+hr)
anal/urinary muscle study
anoscopy diagnostic
ap addendum report
ap ancillary testing report
arterial blood gases
arterial exam lower extremities
arthrocentesis/inject,interm joint/bursa
arthrocentesis/inject,major joint/bursa
bi breast us limited left
bi breast us limited right
biometry
biometry non-surgical eye
biopsy prostate; needle or punch single or multi
biopsy skin add'l lesion
body fluid report
bone density
bone marrow aspiration
bone marrow biopsy
bone marrow report
br diagnostic digital mammogram bilat
br diagnostic digital mammogram lt
br diagnostic digital mammogram rt
br screening dbt mammogram bilat
br screening digital mammogram bilat
br ultrasound breast bilat
br ultrasound guided core biopsy lt
br ultrasound guided core biopsy rt
breast imaging addendum
breath hydrogen test
bronchodilator resp

### names unique to case

In [13]:
case = '\n'.join(sorted(lab_309_frequent_nm - lab_nonSD_frequent_nm))

with open('309_labs_case.txt', 'w') as fp:
    fp.write(case)

!cat 309_labs_case.txt

% of total
alpha-tocopherol
angle native bld prt sulf
angle native bld w protamine sulfate
angle native blood w amicar
anti xa assay-unfractionated heparin
bedside hemoglobin
beta m-spike
cord arterial base ex/def
cord venous base ex/def
estradiol
factor x assay
ft4
glucose 1 hour ob screen 50g
glucose 1hr pp
iga kappa
k native bld w protamine sulfate
k native bld w protm sulf
k native blood w amicar
lithium level
lithium order
ma native bld w prot sulf
ma native bld w protamine sulfate
percent motility
r native bld w protamine sulfate
r native bld w protm sulf
salicylate
sperm concentration
testosterone % free
tobramycin
tsh
viscosity
viscosity plasma
vitamin a (retinol)
vitamin d 1 25 di-oh
volume

In [14]:
case = '\n'.join(sorted(med_309_frequent_nm - med_nonSD_frequent_nm))

with open('309_meds_case.txt', 'w') as fp:
    fp.write(case)

!cat 309_meds_case.txt

acetic acid
amiloride
amphotericin b lipid complex
atomoxetine
belladonna-opium
benztropine
bismuth subsalicylate
bortezomib
buprenorphine-naloxone
buspirone
butalbital-codeine-caffeine-acetaminophen
caffeine-acetaminophen
capsaicin
carboprost
carisoprodol
caspofungin
cefixime
chlordiazepoxide
chloroprocaine
chlorpromazine
chorionic gonadotropin
clomiphene
clozapine
colistimethate
cytarabine
darifenacin
darunavir
dasatinib
desvenlafaxine
dextroamphetamine
dolutegravir
dornase alfa
doxylamine-pyridoxine
efavirenz
estradiol-etonogestrel
etravirine
fa-prenatal multivit-fe-min
famciclovir
fluphenazine
fluticasone propionate
fluvoxamine
follicle stimulating hormone-luteinizing hormone
galantamine
hydrocodone-ibuprofen
ifosfamide
imipramine
insulin aspart-insulin aspart protamine
ipratropium nasal
isopropyl alcohol
lactobacillus acidophilus and bulgaricus
lamivudine-zidovudine
linaclotide
liposomal amphotericin b
lithium
lithium carbonate
lubiprostone
lurasidone
magnesium chloride
memantine 

In [15]:
case = '\n'.join(sorted(procedure_309_frequent_nm - procedure_nonSD_frequent_nm))

with open('309_procedures_case.txt', 'w') as fp:
    fp.write(case)

!cat 309_procedures_case.txt

aerosol inhalation treatment
amniocentesis
anal pressure record
artificial insemination
assess lth/behave initial
behavior change smoking < 3 minutes
biopsy of uterus lining
catheter insertion,indwelling temp
chorion biopsy
cryopreservation sperm
doppler echo exam heart
echo heart 2d f/u limited
fetal non-stress test
flu vaccine whole im
fluoro hysterosalpingogram
gyn ultrasound (pac clinic)
h1n1 influenza vacc inj
hysterosalpingogram
injection bupivicaine hydro
ir cva exch tunnelled
ir ivc filter
ir lp diagnostic
ir lp for chemo inj
kenalog 10 mg intralesional
medical forms simple
methylprednisolone 40 mg inj
moderate sedation by same phys 5 yrs +
mr spine lumbar sacral w contrast
nbuvb hand and foot
neurobehavioral status exam
neuropsych testing by tech
neuropsych tst by psych/phys
nmct lung w inf
ob ultrasound (non nmff)
ob us < 14 wks single fetus
ob us > 14 wks single fetus
ob us early dates
ob us limited greater than 1/2 hr
obtaining screen pap smear
pet brain alzheimer's
pulse o

### names unique to control

In [16]:
control = '\n'.join(sorted(lab_nonSD_frequent_nm - lab_309_frequent_nm))

with open('309_labs_control.txt', 'w') as fp:
    fp.write(control)

!cat 309_labs_control.txt

% free psa
%viability
absolute reticulocyte
activated clotting time
alcohol
aldolase
aldolaseso
alpha 1 antitrypsin
alpha-1 antitrypsin
ant-dna antibody
anti streptolysin
anti-dna antibody
antinuclear antibodies
art fio2%
arterial % hbco
arterial % hbmet
arterial % hbo2
arterial %hbco
arterial o2 content
b cell
be (ecf) arterial poc
bili indirect
bilirubin neonatal
bk virus quant viral load blood
bk virus quant viral load urine
blood
blood gas hematocrit
bnp
ca19-9
ca27.29
carbamazepine
cd3
cells counted
chco3 arterial poc
chco3 cord art poc
cholesterol ldl direct
chromogranin a
ck mb %
collagen/adp
cord arterial bicarbonate
cord arterial pco2
cord arterial ph
cord arterial po2
cord venous bicarbonate
cord venous pco2
cord venous ph
cord venous po2
crossmatch expiration
csf rbc
cso2 arterial poc
cso2 cord ven poc
ctco2 arterial poc
ctco2 cord ven poc
cyclic citrullinated peptide (ccp) ab
cyclosporin
cytomegalovirus viral load pcr
dsdna autoantibodies
egfr(african american,est)
egfr(non

In [17]:
control = '\n'.join(sorted(med_nonSD_frequent_nm - med_309_frequent_nm))

with open('309_meds_control.txt', 'w') as fp:
    fp.write(control)

!cat 309_meds_control.txt

-cyclodeca(3,4)benz(1,2-4,11-2alpha,12aalpha,12balpha))-bis(acetyloxy)-2a,3,4,4a,5,6,9,10,11,12,12a,12b-dodecahydro-5-9-1h-benzenepropanoic acid beta-yl ester (2ar-dihydroxy-7,11-4a,8,13,13-(2aalpha,4beta,4abeta,6beta,9alpha(alphar*,betas*),11alpha,1-alpha-12-(benzoyloxy)- 6,12b-b)oxet-oxo-methano-(benzoylamino)-tetramethyl-hydroxy
1a-interferon beta
acitretin
adapalene-benzoyl peroxide
alemtuzumab
amlodipine-olmesartan
amlodipine-valsartan
ascorbic acid-sodium ascorbate-polyethylene glycol 3350-potassium chloride-sodium sulfate-sodium chloride
atropine-pb-hyoscyamine-scopolamine
azacitidine
benazepril-hydrochlorothiazide
benzocaine otic-antipyrine
bevacizumab
bisoprolol-hydrochlorothiazide
bivalirudin
brimonidine-timolol
c-folic acid-b complex
cabergoline
calcium-magnesium-vitamin d
canagliflozin
chickenpox vaccine
chloroquine
chlorpheniramine-hydrocodone
chondroitin-glucosamine
chondroitin-methylsulfonylmethane-glucosamine
cidofovir
cinacalcet
cranberry preparation
cyclodeca(3,4)benz

In [18]:
control = '\n'.join(sorted(procedure_nonSD_frequent_nm - procedure_309_frequent_nm))

with open('309_procedures_control.txt', 'w') as fp:
    fp.write(control)

!cat 309_procedures_control.txt

2d echocardiogram with doppler
abdomen 1 view
ablate heart dysrhythm focus for svt
admin vac (< 18yr w edu) 1st-component any route
application of forearm cast
arthrocentesis/inject,small joint/bursa
athletic train eval
aud audiological evaluation (comprehensive)
bcg live intravesical vac
bil standing ap view of knees
bladder irrigation or instillation
br diagnostic dbt mammogram bilat
br diagnostic dbt mammogram lt
br digital mammogram needle local lt
br digital mammogram needle local rt
br screening digital mammogram lt
br screening digital mammogram rt
br stereotactic breast core biopsy lt
br stereotactic breast core biopsy rt
breast surgery report
cardiopulmonary exercise stress test (cpet)
cardiopulmonary stress test
cardioversion
carotid doppler
chemo anti-neopl sq/im
chemo hormon antineopl sq/im
collect blood venous device
colon cancer screen low risk (clsma)
colonoscopy w/ biopsy single/multiple
colonoscopy w/biopsy (clsma)
colorectal screen high risk (clsma)
ct abdomen & pelvi

## lab feature matrix

In [19]:
def select_rows(df):
    return (  (df.lab_val < 9999999.0).values
            & (df.lab_ref_high_val_numeric < 9999999.0).values
            & (df.lab_ref_low_val_numeric < 9999999.0).values
            & (df.lab_ref_low_val_numeric < 9999999.0).values
            & (df.lab_nm_lower_case.isin(lab_frequent_nm)).values
           )
# lab_nonSD_frequent_nm | lab_309_frequent_nm
# df = pd.concat([lab_309_sample[lab_309_sample.lab_nm_lower_case.isin(lab_309_frequent_nm)], 
#                 lab_nonSD[lab_nonSD_frequent_nm]])

# set(lab_309_sample[select_rows].columns) - 


df = pd.concat([lab_309_sample, lab_nonSD])
df = df[select_rows]
high_flag = df['lab_val'] > df['lab_ref_high_val_numeric']
low_flag = df['lab_val'] < df['lab_ref_low_val_numeric'] 
df = df.assign(Lab_high=high_flag, Lab_low=low_flag)

lab_feature_mat = df[['person_id', 'lab_nm_lower_case', 'Lab_high', 'Lab_low']]. \
                  groupby(['person_id', 'lab_nm_lower_case']).max().unstack().fillna(False)
lab_feature_mat.columns = ['_'.join(col_name) for col_name in lab_feature_mat.columns]

print '\n lab feature:'
print lab_feature_mat.shape
display(lab_feature_mat.head(10))

pickle.dump(lab_feature_mat, open('lab_feature_mat.p', 'wb'))


 nonSD_lab feature:
(22947, 1140)


,Lab_high_% free psa,Lab_high_% of total,Lab_high_%cd3,Lab_high_%viability,Lab_high_abs cd3+,Lab_high_abs cd3+ cd4+,Lab_high_abs cd3+ cd8+,Lab_high_absolute bands,Lab_high_absolute basophils,Lab_high_absolute blast,Lab_high_absolute cd3,Lab_high_absolute cd3+,Lab_high_absolute cd3/cd4,Lab_high_absolute cd3/cd8,Lab_high_absolute eosinophils,Lab_high_absolute immature granulocytes,Lab_high_absolute lymphocytes,Lab_high_absolute metamyelocyte,Lab_high_absolute monocyte,Lab_high_absolute myelocyte,Lab_high_absolute neutrophil bands,Lab_high_absolute neutrophil metamyelocytes,Lab_high_absolute neutrophil myelocytes,Lab_high_absolute neutrophil promyelocytes,Lab_high_absolute neutrophils,Lab_high_absolute plasma cell,Lab_high_absolute promyelocyte,Lab_high_absolute reticulocyte,Lab_high_absolute segs,Lab_high_activated clotting time,Lab_high_adrenocorticotropic hormone,Lab_high_agap,Lab_high_albumin,Lab_high_albumin (spep),Lab_high_albumin glob,Lab_high_alcohol,Lab_high_aldolase,Lab_high_aldolaseso,Lab_high_alkaline phosphatase,Lab_high_alpha 1 antitrypsin,Lab_high_alpha fetoprotein tumor,Lab_high_alpha-1 antitrypsin,Lab_high_alpha-tocopherol,Lab_high_alt (sgpt),Lab_high_ammonia,Lab_high_angiotensin i converting enzyme,Lab_high_angle native bld prt sulf,Lab_high_angle native bld w protamine sulfate,Lab_high_angle native blood,Lab_high_angle native blood w amicar,...,Lab_low_urine glucose,Lab_low_urine ketones,Lab_low_urine ph,Lab_low_urine potassium,Lab_low_urine protein,Lab_low_urine protein random,Lab_low_urine protein screen,Lab_low_urine rbcs,Lab_low_urine sodium,Lab_low_urine spec gravity,Lab_low_urine specific gravity,Lab_low_urine specific gravity nlfh,Lab_low_urine total protein,Lab_low_urine urea nitro (random),Lab_low_urine urea nitrogen,Lab_low_urine urobilinogen,Lab_low_urine wbcs,Lab_low_valproate,Lab_low_vancomycin,Lab_low_vancomycin dose time,Lab_low_varicella igg,Lab_low_venous % hbco,Lab_low_venous %hbo2,Lab_low_venous base ex/def,Lab_low_venous bicarbonate,Lab_low_venous pco2,Lab_low_venous ph,Lab_low_venous po2,Lab_low_venous total hemoglobin,Lab_low_viscosity,Lab_low_viscosity plasma,Lab_low_vitamin a (retinol),Lab_low_vitamin b1,Lab_low_vitamin b12,Lab_low_vitamin d 1 25 di-oh,Lab_low_vitamin d 25 hydroxy,Lab_low_volume,Lab_low_wb ck-mb (poct),Lab_low_wb inr (poct),Lab_low_wb inr(poct),Lab_low_wb protime (poct),Lab_low_wbc,Lab_low_white cell count,Lab_low_whole blood glucose,Lab_low_whole blood ionized calcium,Lab_low_whole blood ionized calcium c,Lab_low_whole blood potassium,Lab_low_whole blood sodium,Lab_low_zinc serum,Lab_low_zip
person_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
112152,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
112153,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,True,True,False,False
112167,False,False,False,False,False,False,False,False,False,False,False,False,Fals

## med feature matrix

In [1]:
med_309_sample

NameError: name 'med_309_sample' is not defined

In [20]:
def select_rows(df):
    return df.generic_nm_lower_case.isin(med_frequent_nm)
           
df = pd.concat([med_309_sample, med_nonSD])
df = df[select_rows]

med_feature_mat = pd.DataFrame(df[['person_id', 'generic_nm_lower_case', 'patient_medications_id']] \
                               .groupby(['person_id', 'generic_nm_lower_case']).size(), 
                               columns=['Med'])
med_feature_mat = med_feature_mat.applymap(lambda x: 1 if x > 0 else 0).unstack(fill_value=0)
med_feature_mat.columns = ['_'.join(col_name) for col_name in med_feature_mat.columns]

print '\n med feature:'
print med_feature_mat.shape
display(med_feature_mat.head(10))

pickle.dump(med_feature_mat, open('med_feature_mat.p', 'wb'))


 med feature:
(28851, 884)


,"Med_-cyclodeca(3,4)benz(1,2-4,11-2alpha,12aalpha,12balpha))-bis(acetyloxy)-2a,3,4,4a,5,6,9,10,11,12,12a,12b-dodecahydro-5-9-1h-benzenepropanoic acid beta-yl ester (2ar-dihydroxy-7,11-4a,8,13,13-(2aalpha,4beta,4abeta,6beta,9alpha(alphar*,betas*),11alpha,1-alpha-12-(benzoyloxy)- 6,12b-b)oxet-oxo-methano-(benzoylamino)-tetramethyl-hydroxy","Med_-furo(3',4':6,7)naphtho(2,3-d)-glucopyranosyl)oxy)-1,3-5,8,8a,9-tetrahydro-dioxol-3,5-ethylidene-one 9-5-6(5ah)-dimethoxyphenyl)-(5alpha,5abeta,8aalpha,9beta(r*)))-(4-beta-d-o- (5r-((4,6-hydroxy","Med_-methyl-nitro-(2-1,1-2-furanyl)methyl)thio)ethyl)-ethenediamine n-((dimethylamino)methyl)-(((5-n'",Med_1a-interferon beta,Med_3 acid ethyl esters (usp)-omega,Med_3 fatty acids-omega,Med_abatacept,Med_acetaminophen,Med_acetazolamide,Med_acetic acid,Med_acetylcysteine,Med_acitretin,Med_activated dimethicone,Med_activated methylpolysiloxane,Med_acyclovir,Med_adalimumab,Med_adapalene,Med_adapalene-benzoyl peroxide,Med_adenosine,Med_adol-tetanus-pertuss (tdap) adult-diphth,Med_aflibercept,Med_albumin human usp,Med_albuterol,Med_albuterol-ipratropium,Med_alclometasone,Med_alemtuzumab,Med_alendronate,Med_alendronic acid,Med_alfentanil,Med_alfuzosin,Med_allopurinol,Med_alprazolam,Med_alteplase,Med_amantadine,Med_amikacin,Med_amiloride,Med_aminocaproic acid-6,Med_aminophylline,Med_amiodarone,Med_amitriptyline,Med_amlodipine,Med_amlodipine-atorvastatin,Med_amlodipine-benazepril,Med_amlodipine-olmesartan,Med_amlodipine-valsartan,Med_amoxicillin,Med_amoxicillin-clavulanate,Med_amphetamine-dextroamphetamine,Med_amphotericin b lipid complex,Med_amphotericin b liposome,...,Med_tretinoin,Med_triamcinolone,Med_triamcinolone nasal,Med_triamterene,Med_triiodothyronine,Med_trimethoprim-polymyxin b,Med_trimethoprim-sulfamethoxazole,Med_tropicamide,Med_trospium,Med_turmeric extract,Med_typhoid vaccine live,Med_ubiquinone,Med_unknown,Med_urea,Med_ursodeoxycholate,Med_ursodiol,Med_valacyclovir,Med_valent conjugate vaccine-pneumococcal 13,Med_valganciclovir,Med_valproate,Med_valproic acid,Med_valsartan,Med_vancomycin,Med_vardenafil,Med_varenicline,Med_varenicline tartrate,Med_vasopressin (usp),Med_vedolizumab,Med_venlafaxine,Med_verapamil,Med_vilazodone,Med_vincristine,Med_vit d-calcium carbonate-min,Med_vitamin a,Med_vitamin b 12,Med_vitamin b6,Med_vitamin d,Med_vitamin e,Med_vitamin k 1,Med_vitamins a d and e,Med_voriconazole,Med_warfarin,Med_zaleplon,Med_zinc oxide,Med_zinc sulfate,Med_ziprasidone,Med_zoledronic acid,Med_zolmitriptan,Med_zolpidem,Med_zonisamide
person_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
112152,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
112153,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
112167,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
112206,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
112331,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
112405,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
112488,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

## procedure feature matrix

In [21]:
def select_rows(df):
    return (  (df.order_type_desc != 'LAB').values
            & (df.order_type_desc != 'VASCULAR LAB').values
            & (df.order_nm_lower_case.isin(procedure_frequent_nm)).values
           )
           
df = pd.concat([procedure_309_sample, procedure_nonSD])[select_rows]

procedure_feature_mat = pd.DataFrame(df.groupby(['person_id', 'order_nm_lower_case']).size(), 
                                     columns=['Procedure']).applymap(lambda x: 1 if x > 0 else 0).unstack(fill_value=0)
procedure_feature_mat.columns = ['_'.join(col_name) for col_name in procedure_feature_mat.columns]

print '\n procedure feature:'
print procedure_feature_mat.shape
display(procedure_feature_mat.head(10))

pickle.dump(procedure_feature_mat, open('procedure_feature_mat.p', 'wb'))


 procedure feature:
(26272, 782)


,Procedure_2d echocardiogram with doppler,Procedure_abdomen 1 view,Procedure_ablate heart dysrhythm focus for svt,Procedure_acoustic reflex testing,Procedure_adenosine (vasodilator) perfusion study,Procedure_admin vac (< 18yr w edu) 1st-component any route,Procedure_aerosol inhalation treatment,Procedure_allergy patch tests,Procedure_allergy prick tests for aeros,Procedure_allergy(intradermal)skin tests,Procedure_ambulatory bp monitoring (24+hr),Procedure_amniocentesis,Procedure_anal pressure record,Procedure_anal/urinary muscle study,Procedure_anoscopy diagnostic,Procedure_ap addendum report,Procedure_ap ancillary testing report,Procedure_application of forearm cast,Procedure_arterial blood gases,Procedure_arterial exam lower extremities,"Procedure_arthrocentesis/inject,interm joint/bursa","Procedure_arthrocentesis/inject,major joint/bursa","Procedure_arthrocentesis/inject,small joint/bursa",Procedure_artificial insemination,Procedure_assess lth/behave initial,Procedure_athletic train eval,Procedure_aud audiological evaluation (comprehensive),Procedure_bcg live intravesical vac,Procedure_behavior change smoking < 3 minutes,Procedure_bi breast us limited left,Procedure_bi breast us limited right,Procedure_bil standing ap view of knees,Procedure_biometry,Procedure_biometry non-surgical eye,Procedure_biopsy of uterus lining,Procedure_biopsy prostate; needle or punch single or multi,Procedure_biopsy skin add'l lesion,Procedure_bladder irrigation or instillation,Procedure_body fluid report,Procedure_bone density,Procedure_bone marrow aspiration,Procedure_bone marrow biopsy,Procedure_bone marrow report,Procedure_br diagnostic dbt mammogram bilat,Procedure_br diagnostic dbt mammogram lt,Procedure_br diagnostic digital mammogram bilat,Procedure_br diagnostic digital mammogram lt,Procedure_br diagnostic digital mammogram rt,Procedure_br digital mammogram needle local lt,Procedure_br digital mammogram needle local rt,...,Procedure_xr knee ap/lat/obls/sunrise right,Procedure_xr knee ap/lat/obls/tunnel left,Procedure_xr knee ap/lat/obls/tunnel right,Procedure_xr knee ap/lat/sunrise left,Procedure_xr knee ap/lat/sunrise right,Procedure_xr knee ap/lateral left,Procedure_xr knee ap/lateral right,Procedure_xr knee ap/lateral/obliques left,Procedure_xr knee ap/lateral/obliques right,Procedure_xr knee notch view bilateral,Procedure_xr knees standing bilateral,Procedure_xr knees standing bilateral ap lateral,Procedure_xr leg (tibula/fibula) left,Procedure_xr leg (tibula/fibula) right,Procedure_xr leg length,Procedure_xr leg tibia/fibula right 2 views,Procedure_xr neck (soft tissue),Procedure_xr panorex,Procedure_xr pelvis ap (1 view),Procedure_xr ribs left (w/ 1 vw chest),Procedure_xr ribs left (w/chest),Procedure_xr ribs right (w/chest),Procedure_xr shoulder series left (2 view min),Procedure_xr shoulder series left 3 views,Procedure_xr shoulder series right (2 view min),Procedure_xr shoulder series right 3 views,Procedure_xr spine cervical (1 view),Procedure_xr spine cervical (ap/lat/flex/ext),Procedure_xr spine cervical (ap/lateral),Procedure_xr spine cervical (flex/ext only),Procedure_xr spine cervical (full series),Procedure_xr spine cervical (full series+flex/ext),Procedure_xr spine cervical full series 5 views,Procedure_xr spine lumbar (ap/lateral),Procedure_xr spine lumbar (ap/lateral/flex/ext),Procedure_xr spine lumbar (full series),Procedure_xr spine lumbar full series 5 views,Procedure_xr spine sacrum/coccyx,Procedure_xr spine scoliosis (ap/lateral 14x36),Procedure_xr spine si joints,Procedure_xr spine thoracic (3 views),Procedure_xr spine thoracic (ap/lateral),Procedure_xr toe 1st (great) left,Procedure_xr wrist left,Procedure_xr wrist left (3 view),Procedure_xr wrist right,Procedure_xr wrist right (3 view min),Procedure_xr wrist right (3 view),Procedure_yellow fever vaccine sc,Procedure_zoster vacc sc
person_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
112152,0,0,0,0,1,0,0,